In [ ]:
%pip install pandas

: 

In [21]:
import pandas as pd
import numpy as np

In [ ]:
# ===Base de dados===

# Informações sobre cálculos, etc.
# https://www.tre-ce.jus.br/comunicacao/noticias/2022/Agosto/entenda-como-funiona-o-calculo-dos-quocientes-partidario-e-eleitoral

# Conjuntos de dados
# https://sig.tse.jus.br/ords/dwapr/r/seai/sig-eleicao-arquivo/conjuntos-de-dados?p10_cd_modulo=resultado&session=309649826689846&cs=1iWGX_4kCm1I9RFT7LM3g8OGvjyuhSs0dfNeA6vcgSvnZ5DWoq10rybAKreqCloli3ZA2UxYgg9qMOPODXrRUuA

votacao_partido_df = pd.read_csv('votacao_partido.csv', delimiter=';', encoding='latin_1')

votacao_df = pd.read_csv('votacao_candidato.csv', delimiter=';', encoding='latin_1')

In [23]:
# Subtraindo a quantidade de votos da deputada do total de votos válidos
votacao_df['Votos válidos'] = votacao_df['Votos válidos'].apply(
    lambda x: x - votacao_df['Votos nominais'].loc[votacao_df['Nome candidato'] == 'CARLA ZAMBELLI SALGADO DE OLIVEIRA'])

# votos Zambelli = 946244
# Eliminando a candidata do quadro de candidatos
votacao_df = votacao_df.drop(votacao_df.loc[votacao_df['Nome candidato'] == 'CARLA ZAMBELLI SALGADO DE OLIVEIRA'].index)
votacao_df.head()

,Ano de eleição,Cargo,UF,Nome candidato,Número candidato,Partido,Gênero,Votos válidos,Votos nominais,Data de carga
0,2022,Deputado Federal,SP,ELAINE CRISTINA DO NASCIMENTO,1082,REPUBLICANOS,Feminino,22429341,3185,2025-03-25 03:36:58
1,2022,Deputado Federal,SP,WESLEY RENATO DE OLIVEIRA SOUZA,2280,PL,Masculino,22429341,15595,2025-03-25 03:36:58
2,2022,Deputado Federal,SP,CLAUDINEI ALEXANDRE PIRES,7051,AVANTE,Masculino,22429341,2532,2025-03-25 03:36:58
3,2022,Deputado Federal,SP,RICARDO DE DEUS ANDRADE,3671,AGIR,Masculino,22429341,66,2025-03-25 03:36:58
4,2022,Deputado Federal,SP,CÃNDIDO ELPÍDIO DE SOUZA VACCAREZZA,7070,AVANTE,Masculino,22429341,6397,2025-03-25 03:36:58


Precisamos agora calcular a nova divisão de vagas entre os partidos

In [ ]:
votacao_partido_df['Quantidade de votos válidos'].loc[votacao_partido_df['Sigla partido'] == 'PL']

# O partido PL obteve 5343667 votos, subtraindo o número de votos da candidata do total, temos 4397423
votos_pl = 5343667 - 946244

# Atribuindo a nova quantidade de votos válidos ao partido da candidata
votacao_partido_df['Quantidade de votos válidos'].loc[votacao_partido_df['Sigla partido'] == 'PL'] = votos_pl
votacao_partido_df

In [25]:
# São 70 vagas para Deputado Federal, precisamos também recalcular o quociente eleitoral
# 22429341 / 70 = 320420
quociente_eleitoral = 22429341 / 70

# Calculando o novo quociente partidário
votacao_partido_df = votacao_partido_df.assign(quociente_partidario=lambda x: x['Quantidade de votos válidos'] // quociente_eleitoral)

# 58 vagas foram preenchidas através do quociente partidário, restando 12 vagas para cálculo das sobras
votacao_partido_df['quociente_partidario'].sum()

np.float64(58.0)

In [ ]:
# Esse loop executa o cálculo da média do partido, altera o valor do quociente partidário 
# e faz o cálculo da nova média a cada inserção na média partidária preenchendo as 12 vagas
# de 'sobras'
for i in range(59, 70+1):
    
    # Média mínima para concorrer as vagas de 'sobras'
    # 320420 * 0,8
    media_minima = quociente_eleitoral * 0.8

    # Cria nova coluna com a média dos partidos
    votacao_partido_df = votacao_partido_df.assign(media_sobra=lambda x: x['Quantidade de votos válidos'] / (x['quociente_partidario'] + 1))

    sobra = votacao_partido_df['Sigla partido'].loc[votacao_partido_df['media_sobra'].idxmax()]
    media_partido = votacao_partido_df['media_sobra'].max()
    
    # Apenas os partidos que alcançaram 80% do quociente eleitoral concorrem as vagas de 'sobras'
    if media_partido > media_minima:
        
        # Atualizando o quociente partidário para acrescentar o número de vagas de 'sobras'
        votacao_partido_df['quociente_partidario'].loc[votacao_partido_df['Sigla partido'] == sobra] = votacao_partido_df['quociente_partidario'].loc[votacao_partido_df['Sigla partido'] == sobra].apply(lambda x: x + 1)


In [ ]:
# Divisão final das vagas entre os partidos
votacao_partido_df

com o quadro de vagas por partido em mãos, vamos agora preencher as cadeiras com os candidatos eleitos

In [ ]:
# Aplicando ordem decrescente por votos nominais
votacao_df = votacao_df.sort_values(by='Votos nominais', ascending=False)
votacao_df.loc[votacao_df['Partido'] == 'PSOL'].head(5)

,Ano de eleição,Cargo,UF,Nome candidato,Número candidato,Partido,Gênero,Votos válidos,Votos nominais,Data de carga
1415,2022,Deputado Federal,SP,GUILHERME CASTRO BOULOS,5010,PSOL,Masculino,22429341,1001472,2025-03-25 03:36:58
460,2022,Deputado Federal,SP,ERIKA SANTOS SILVA,5070,PSOL,Feminino,22429341,256903,2025-03-25 03:36:58
35,2022,Deputado Federal,SP,SÂMIA DE SOUZA BOMFIM,5000,PSOL,Feminino,22429341,226187,2025-03-25 03:36:58
1337,2022,Deputado Federal,SP,SÔNIA BONE DE SOUSA SILVA SANTOS,5088,PSOL,Feminino,22429341,156966,2025-03-25 03:36:58
561,2022,Deputado Federal,SP,LUIZA ERUNDINA DE SOUSA,5021,PSOL,Feminino,22429341,113983,2025-03-25 03:36:58
